## Introdução do problema
Neste notebook será realizada a seleção dos melhores regressores para analise de sentimento obtidos através de letras musicais. com os resultados poderemos ver os valores de sentimentos positivos negativos ou neutros de cada musica. estes valores serão os tutores para os regressores.

## Principal atributo a ser trabalhado

lyrics: Letra completa da música</br>

## Importação das bibliotecas 

In [1]:
!pip install vaderSentiment
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
from textblob import TextBlob
import gdown
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

from sklearn import preprocessing
from sklearn import utils
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor


## Metodologias


A metodologia a ser utilizada neste trabalho é baseada na metodologia de outros dois trabalhos

o primeiro é o [Metodologias para Análise de Sentimentos de Tweets
sobre o Mercado Financeiro](https://bdm.unb.br/bitstream/10483/29207/1/2019_MuriloCerqueiraMedeiros_tcc.pdf). A metodologia apresentada neste TCC é descrita da seguinte forma: primeiramente, os tweets são carregados de um arquivo em disco, e são pré-processados, gerando-se um espaço vetorial. Em seguida, ocorre a extração de tópicos e a redução de dimensionalidade em paralelo. O resultado da redução de dimensionalidade é utilizado para classificar os sentimentos, para agrupar os tweets e para análise visual, a fim de tornar a investigação de aspectos em comum nos dados mais objetiva.

o segundo é o [Estudo e avaliação de métodos de análise de sentimentos baseada em aspectos para textos opinativos em português](https://www.teses.usp.br/teses/disponiveis/59/59143/tde-31102018-174346/publico/mateuscorrigida.pdf). A metodologia utilizada nesta tese é dividida em duas etapas, na primeira é identificada as caracteristicas do produto avaliado mencionados no texto, na segunda o foco é identificar a polaridade (positiva ou negativa) relacionado a cada aspecto do produto.

A princípio, neste trabalho é coletada a base de dados, realizado o pré-processamento da mesma, realizada a mineração do sentimento utilizando a biblioteca Vader, gerado a média dos valores agrupados por gênero, e comparação dos sentimentos entre os gêneros.

## Desenvolvimento

### Gdown pra baixar a base de dados

Aqui é definido o caminho no drive onde esta localizado o arquivo com a base de dados

In [ ]:
url = "https://drive.google.com/uc?id=1UgsRtv23jTZ4p2fCfe9Ccl3ASVMTTLLM"
output = "dataset_musicas.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UgsRtv23jTZ4p2fCfe9Ccl3ASVMTTLLM
To: /content/dataset_musicas.csv
100%|██████████| 27.7M/27.7M [00:00<00:00, 111MB/s]


'dataset_musicas.csv'

### Carregamento da base de dados

In [ ]:
dataset = pd.read_csv("/content/dataset_musicas.csv")

In [ ]:
dataset.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


### Pre-processamento

#### Remoção das colunas que não serão utilizadas

In [ ]:
dataset.columns.size

31

In [ ]:
dataset.drop(columns=["release_date","len", 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age', 'artist_name'], inplace=True)

In [ ]:
dataset.head()

,Unnamed: 0,track_name,genre,lyrics
0,0,mohabbat bhi jhoothi,pop,hold time feel break feel untrue convince spea...
1,4,i believe,pop,believe drop rain fall grow believe darkest ni...
2,6,cry,pop,sweetheart send letter goodbye secret feel bet...
3,10,patricia,pop,kiss lips want stroll charm mambo chacha merin...
4,12,apopse eida oneiro,pop,till darling till matter know till dream live ...


#### Remoção das stopwords

Ao analisar o conteúdo de algumas letras pertecentes ao dataset, percebi que as stopwords já foram removidas, desta forma o texto ja está pronto para ser enviado para o analisador de sentimento

### Analise de sentimento

In [ ]:
analyzer = SentimentIntensityAnalyzer()
df = pd.DataFrame(columns=["nome","genero","letra","positivo","negativo","neutro","compound"])

for index, row in dataset.iterrows():
  analisado = analyzer.polarity_scores(row['lyrics'])
  neu = analisado['neu']
  pos = analisado['pos']
  neg = analisado['neg']
  compound = analisado['compound']
  df = df.append({
      'nome' : row["track_name"], 
      'genero' : row["genre"], 
      'letra' : row["lyrics"],
      'positivo' : pos, 
      'negativo' : neg, 
      'neutro' : neu,
      'compound' : compound, 
      }, ignore_index=True)

df.head()



In [ ]:
df.shape

(28372, 7)

#### Salvamento da base pré-processada com sentimento

In [ ]:
df.to_csv("letras_analise_sentimento.csv")

#### Carregamento da base pré-processada

In [3]:
url = "https://drive.google.com/uc?id=1HHDgs5LireyXYAYbzkh5fifwi43-JwS2"
output = "letras_analise_sentimento.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1HHDgs5LireyXYAYbzkh5fifwi43-JwS2
To: /content/letras_analise_sentimento.csv
100%|██████████| 14.2M/14.2M [00:00<00:00, 88.8MB/s]


'letras_analise_sentimento.csv'

In [4]:
df = pd.read_csv("letras_analise_sentimento.csv")

In [5]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.head()

,nome,genero,letra,positivo,negativo,neutro,compound
0,mohabbat bhi jhoothi,pop,hold time feel break feel untrue convince spea...,0.371,0.211,0.418,0.9398
1,i believe,pop,believe drop rain fall grow believe darkest ni...,0.150,0.259,0.591,-0.7717
2,cry,pop,sweetheart send letter goodbye secret feel bet...,0.305,0.000,0.695,0.8481
3,patricia,pop,kiss lips want stroll charm mambo chacha merin...,0.533,0.038,0.429,0.9887
4,apopse eida oneiro,pop,till darling till matter know till dream live ...,0.415,0.128,0.458,0.9623


#### Separação da base em treino e teste

In [6]:
vect_word = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
                        stop_words= 'english',dtype=np.float32)

In [7]:
X = df['letra']
y = df[['positivo','negativo','neutro']]

In [8]:
X = vect_word.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.2)

#### definição dos regressores

In [9]:
ln = MultiOutputRegressor(LinearRegression())
dtr = MultiOutputRegressor(DecisionTreeRegressor())
mlpr = MultiOutputRegressor(MLPRegressor())

#### treinamento e score de treinamento de cada regressor

In [ ]:
#Treinamento arvore decisão
%time dtr.fit(X,y)

CPU times: user 1min 12s, sys: 21.8 ms, total: 1min 12s
Wall time: 1min 12s


MultiOutputRegressor(estimator=DecisionTreeRegressor())

In [ ]:
#score de treinamento
dtr.score(X,y)

0.9999556414353725

In [ ]:
#Treinamento regressão linear
%time ln.fit(X,y)

CPU times: user 682 ms, sys: 3.03 ms, total: 685 ms
Wall time: 693 ms


MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
#score de treinamento
ln.score(X,y)

0.7782661286102168

In [ ]:
#Treinamento mlp
%time mlpr.fit(X,y)

CPU times: user 1min 34s, sys: 1min 21s, total: 2min 56s
Wall time: 1min 31s


MultiOutputRegressor(estimator=MLPRegressor())

In [ ]:
#score de treinamento
mlpr.score(X,y)

0.9906137667712863

### Metricas

As métricas para se avaliar os regressores serão as seguintes:

**MSE:** é o Erro Quadrático Médio, é uma Métrica muito utilizada,  consiste na média do erro das previsões ao quadrado. Em outras palavras, pega-se a diferença entre o valor predito pelo modelo e o valor real, eleva-se o resultado ao quadrado, faz-se a mesma coisa com todos os outros pontos, soma-os, e dividi-se pelo número de elementos preditos. Quanto mais próximo de 0 melhor o modelo.

**MAE:** É o Erro Absoluto Médio, consiste na média das distâncias entre valores preditos e reais. Diferentemente do MSE e do RMSE, essa métrica não “pune” tão severamente os outliers do modelo.
Essa medida apresenta valor mínimo 0 e não apresenta valor máximo, quanto mais próximo de 0 melhor o modelo.

**RMSE:** É a Raiz Quadrática Média, Tendo em vista essa diferença de unidades, o RMSE entra como uma forma de melhorar a interpretabilidade da métrica, acertando a unidade. Entretanto, essa medida, assim como o MSE, penaliza predições muito distantes da real. Quanto mais próximo de 0 melhor o modelo

**Score(R²):** Muito utilizada em modelos de regressões da área de finanças, o R-Quadrado, ou Coeficiente de Determinação, é uma métrica que visa expressar a quantidade da variança dos dados que é explicada pelo modelo construído. Em outras palavras, essa medida calcula qual a porcentagem da variança que pôde ser prevista pelo modelo de regressão e, portanto, nos diz o quão “próximo” as medidas reais estão do nosso modelo.

O valor do seu R-Quadrado varia de 0 a 1 e geralmente é representado em porcentagem. Por exemplo, um R² = 75% nos diz que 75% da variância de nossos dados podem ser explicados pelo modelo construído, enquanto os outros 25%, teoricamente, se tratariam de uma variância residual. Quanto mais próximo de 1 melhor o modelo.

In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
import time
scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_root_mean_squared_error']

In [11]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler 
from sklearn.model_selection import GridSearchCV

Criação dos pipelines

In [12]:
pipe_mlp = make_pipeline(MaxAbsScaler(),MLPRegressor(random_state=42))
mlp = MultiOutputRegressor(pipe_mlp)
rt = MultiOutputRegressor(DecisionTreeRegressor(random_state=42))
forest = MultiOutputRegressor(RandomForestRegressor(random_state=42, n_jobs = -1))
pipe_knn = make_pipeline(MaxAbsScaler(), KNeighborsRegressor())
knn = MultiOutputRegressor(pipe_knn)

In [ ]:
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print('MSE %.5f' % (mean_squared_error(y_test, mlp_pred)))
print('MAE %.5f' % (mean_absolute_error(y_test, mlp_pred)))
print('RMSE %.5f' % (mean_squared_error(y_test, mlp_pred, squared=False)))
print('Score %.3f' % (mlp.score(X_test, y_test)))

scores = cross_validate(mlp, X, y, scoring=scoring, cv=10)
print('CV - MSE %.5f' % (abs(scores['test_neg_mean_squared_error']).mean()))
print('CV - MAE %.5f' % (abs(scores['test_neg_mean_absolute_error']).mean()))
print('CV - RMSE %.5f' % (abs(scores['test_neg_root_mean_squared_error']).mean()))

MSE 0.00513
MAE 0.05185
RMSE 0.07135
Score 0.794
CV - MSE 0.00520
CV - MAE 0.05214
CV - RMSE 0.07165


In [ ]:
rt.fit(X_train, y_train)
rt_pred = rt.predict(X_test)
print('MSE %.5f' % (mean_squared_error(y_test, rt_pred)))
print('MAE %.5f' % (mean_absolute_error(y_test, rt_pred)))
print('RMSE %.5f' % (mean_squared_error(y_test, rt_pred, squared=False)))
print('Score %.3f' % (rt.score(X_test, y_test)))

scores = cross_validate(rt, X, y, scoring=scoring, cv=10)
print('CV - MSE %.5f' % (abs(scores['test_neg_mean_squared_error']).mean()))
print('CV - MAE %.5f' % (abs(scores['test_neg_mean_absolute_error']).mean()))
print('CV - RMSE %.5f' % (abs(scores['test_neg_root_mean_squared_error']).mean()))

MSE 0.01935
MAE 0.10030
RMSE 0.13778
Score 0.226
CV - MSE 0.01968
CV - MAE 0.10169
CV - RMSE 0.13878


In [ ]:
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print('MSE %.5f' % (mean_squared_error(y_test, forest_pred)))
print('MAE %.5f' % (mean_absolute_error(y_test, forest_pred)))
print('RMSE %.5f' % (mean_squared_error(y_test, forest_pred, squared=False)))
print('Score %.3f' % (forest.score(X_test, y_test)))

scores = cross_validate(forest, X, y, scoring=scoring, cv=10)
print('CV - MSE %.5f' % (abs(scores['test_neg_mean_squared_error']).mean()))
print('CV - MAE %.5f' % (abs(scores['test_neg_mean_absolute_error']).mean()))
print('CV - RMSE %.5f' % (abs(scores['test_neg_root_mean_squared_error']).mean()))

MSE 0.00984
MAE 0.07237
RMSE 0.09851
Score 0.613


In [ ]:
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
print('MSE %.5f' % (mean_squared_error(y_test, knn_pred)))
print('MAE %.5f' % (mean_absolute_error(y_test, knn_pred)))
print('RMSE %.5f' % (mean_squared_error(y_test, knn_pred, squared=False)))
print('Score %.3f' % (knn.score(X_test, y_test)))

scores = cross_validate(knn, X, y, scoring=scoring, cv=10)
print('CV - MSE %.5f' % (abs(scores['test_neg_mean_squared_error']).mean()))
print('CV - MAE %.5f' % (abs(scores['test_neg_mean_absolute_error']).mean()))
print('CV - RMSE %.5f' % (abs(scores['test_neg_root_mean_squared_error']).mean()))

MSE 0.01819
MAE 0.10478
RMSE 0.13452
Score 0.277
CV - MSE 0.01882
CV - MAE 0.10588
CV - RMSE 0.13647


#### MLP

In [13]:
parameters_mlp = {'estimator__mlpregressor__solver': ['lbfgs', 'adam', 'sgd'],
              'estimator__mlpregressor__activation': ['logistic', 'relu'],
              'estimator__mlpregressor__learning_rate': ['constant', 'adaptive']}

In [14]:
clf1 = GridSearchCV(estimator=mlp, 
                   param_grid=parameters_mlp,
                   scoring='neg_mean_squared_error',
                   cv=5,
                   n_jobs=-1)
tic = time.perf_counter()
clf1 = clf1.fit(X_train, y_train)
toc = time.perf_counter()
print(f"Busca executada em {toc - tic:0.2f} segundos")
print(clf1.best_params_)
print(clf1.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Busca executada em 2363.37 segundos
{'estimator__mlpregressor__activation': 'relu', 'estimator__mlpregressor__learning_rate': 'constant', 'estimator__mlpregressor__solver': 'lbfgs'}
-0.004491476501257202


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [17]:
y_pred_clf1 = clf1.predict(X_test)
mse_clf1 = mean_squared_error(y_test, y_pred_clf1)
mae_clf1 = mean_absolute_error(y_test, y_pred_clf1)
rmse_clf1 = mean_squared_error(y_test, y_pred_clf1, squared=False)
print('MSE %.5f' % (mse_clf1))
print('MAE %.5f' % (mae_clf1))
print('RMSE %.5f' % (rmse_clf1))

MSE 0.00441
MAE 0.04667
RMSE 0.06609


#### Árvore de decisão

In [18]:
parameters_rt = {'estimator__max_depth': [None,2,3,5,8,10,20,30]}

In [19]:
clf2 = GridSearchCV(estimator=rt, 
                   param_grid=parameters_rt,
                   scoring='neg_mean_squared_error',
                   cv=5,
                   n_jobs=-1)
tic = time.perf_counter()
clf2 = clf2.fit(X_train, y_train)
toc = time.perf_counter()
print(f"Busca executada em {toc - tic:0.2f} segundos")
print(clf2.best_params_)
print(clf2.best_score_)

Busca executada em 304.34 segundos
{'estimator__max_depth': 20}
-0.017809516623174305


In [22]:
y_pred_clf2 = clf2.predict(X_test)
mse_clf2 = mean_squared_error(y_test, y_pred_clf2)
mae_clf2 = mean_absolute_error(y_test, y_pred_clf2)
rmse_clf2 = mean_squared_error(y_test, y_pred_clf2, squared=False)
print('MSE %.5f' % (mse_clf2))
print('MAE %.5f' % (mae_clf2))
print('RMSE %.5f' % (rmse_clf2))

MSE 0.01809
MAE 0.10171
RMSE 0.13366


#### Floresta Aleatória

In [ ]:
parameters_forest = {'estimator__n_estimators':[500, 1000, 1500, 2000, 2500],
                      'estimator__max_depth':[None,2,3,5,8,10,20,30]}

In [ ]:
clf3 = GridSearchCV(estimator=forest, 
                   param_grid=parameters_forest,
                   scoring='neg_mean_squared_error',
                   cv=5,
                   n_jobs=-1)
tic = time.perf_counter()
clf3 = clf3.fit(X_train, y_train)
toc = time.perf_counter()
print(f"Busca executada em {toc - tic:0.2f} segundos")
print(clf3.best_params_)
print(clf3.best_score_)

In [ ]:
y_pred_clf3 = clf3.predict(X_test)
mse_clf3 = mean_squared_error(y_test, y_pred_clf3)
print('MSE %.5f' % (mse_clf3))

#### KNN

In [23]:
parameters_knn = {'estimator__kneighborsregressor__n_neighbors':[5,10,15,20,25,30,35,40]}

In [24]:
clf4 = GridSearchCV(estimator=knn, 
                   param_grid=parameters_knn,
                   scoring='neg_mean_squared_error',
                   cv=5,
                   n_jobs=-1)
tic = time.perf_counter()
clf4 = clf4.fit(X_train, y_train)
toc = time.perf_counter()
print(f"Busca executada em {toc - tic:0.2f} segundos")
print(clf4.best_params_)
print(clf4.best_score_)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Busca executada em 369.81 segundos
{'estimator__kneighborsregressor__n_neighbors': 40}
-0.013491488594176185


In [25]:
y_pred_clf4 = clf4.predict(X_test)
mse_clf4 = mean_squared_error(y_test, y_pred_clf4)
mae_clf4 = mean_absolute_error(y_test, y_pred_clf4)
rmse_clf4 = mean_squared_error(y_test, y_pred_clf4, squared=False)
print('MSE %.5f' % (mse_clf4))
print('MAE %.5f' % (mae_clf4))
print('RMSE %.5f' % (rmse_clf4))

MSE 0.01414
MAE 0.09280
RMSE 0.11860


**Resultados das métricas:**
<br>**antes da variação de parametros** com o grid search, o regressor mlp em todas as métricas se destacou em relação aos outros regressores utilizados(Árvore de Decisão, Floresta Aleatória e KNN). os valores de suas métricas foram:
<br>MSE: 0.005
<br>MAE: 0.05
<br>RMSE: 0.07
<br>Score: 0.79
**Depois da variação de parametros** com o grid search, o regressor mlp em todas as métricas se destacou em relação aos outros regressores utilizados(Árvore de Decisão e KNN). os valores de suas métricas foram:
<br>MSE: 0.004
<br>MAE: 0.04
<br>RMSE: 0.06


Desta forma, para esta análise o MLP é o regressor mais recomendado entre todos os regressores utilizados